<a href="https://colab.research.google.com/github/Simone999/AshtonTablutAgent/blob/master/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Enable GPU

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Nuova sezione

In [ ]:
# !unzip drive/My\ Drive/simulazioni.zip -d simulations

In [80]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
import glob

In [73]:
dir = 'simulations'
files = glob.glob(f'{dir}/simulation_*.csv')
len(files)

1

In [78]:
pawns = {-1:0, 10:0, 0: 1/3, 1:2/3, 100: 1}

def map(item, dict):
  return dict[item]
vmap = np.vectorize(map)

def load_data(filename):
  df = pd.read_csv(filename)
  
  df = df.drop('turns', axis=1)
  boards=list(df['board'])

  boards=[board.replace('[','').replace(']\n','').replace(']]','').replace('1100','1 100').replace('-1100','-1 100').replace('0100','0 100')
          for board in boards]
  boards = [list(vmap(np.fromstring(board, sep=' ', dtype=int), pawns)) for board in boards]
  boards = np.array(boards)

  board_size = boards.shape[1]
  
  data = dict(zip(range(0, board_size), boards.T))
  for column in df.drop('board', axis=1).columns:
    data[column] = df[column].values

  df = pd.DataFrame(data)
  df['winner'] = df['winner'].map(outcomes)

  return pd.DataFrame(data)

def create_network(input_size, output_size):
  model = Sequential()
  model.add(Dense(12, input_dim=input_size, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(output_size, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model



# df_black=df[df['turns']==0]
# df_white=df[df['turns']==1]

### Create Neural Network

In [89]:
input_size = 81
output_size = 3

print('input_size', input_size)
print('output_size', output_size)

model = create_network(input_size, output_size)
model.summary()

input_size 81
output_size 3
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 12)                984       
                                                                 
 dense_56 (Dense)            (None, 8)                 104       
                                                                 
 dense_57 (Dense)            (None, 3)                 27        
                                                                 
Total params: 1,115
Trainable params: 1,115
Non-trainable params: 0
_________________________________________________________________


### Load data

In [ ]:
# for filename in files:
#   df = pf.read_csv(filename)
#   train(df)

filename = files[0]
df = load_data(filename)
df = df.sample(frac=1)

X = df.drop('winner', axis=1)
y = to_categorical(df['winner'], 3)

print(X.shape)

### Train

In [95]:
with tf.device('/device:GPU:0'):
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
  model.fit(X, y, epochs=150, batch_size=10, validation_split=0.2,callbacks=[callback])

model.save('value_model')

Epoch 1/150
3145/3145 [==============================] - 15s 5ms/step - loss: 0.7652 - accuracy: 0.6247 - val_loss: 0.8145 - val_accuracy: 0.6183
Epoch 2/150
3145/3145 [==============================] - 15s 5ms/step - loss: 0.7659 - accuracy: 0.6252 - val_loss: 0.8178 - val_accuracy: 0.6205
Epoch 3/150
3145/3145 [==============================] - 14s 4ms/step - loss: 0.7653 - accuracy: 0.6266 - val_loss: 0.8234 - val_accuracy: 0.5924
Epoch 4/150
3145/3145 [==============================] - 14s 5ms/step - loss: 0.7647 - accuracy: 0.6251 - val_loss: 0.8163 - val_accuracy: 0.6191
Epoch 5/150
3145/3145 [==============================] - 14s 5ms/step - loss: 0.7648 - accuracy: 0.6236 - val_loss: 0.8253 - val_accuracy: 0.5926
Epoch 6/150
3145/3145 [==============================] - 14s 5ms/step - loss: 0.7640 - accuracy: 0.6261 - val_loss: 0.8180 - val_accuracy: 0.6202
Epoch 7/150
3145/3145 [==============================] - 15s 5ms/step - loss: 0.7646 - accuracy: 0.6247 - val_loss: 0.8216 -